# Constitutional Reasoning Experiment - Statistical Deep-Dive

**Purpose:** Rigorous statistical analysis and hypothesis testing

This notebook tests specific hypotheses about constitutional reasoning integrity:
1. Are differences between models statistically significant?
2. Is the bad-faith vs. honest gap significant?
3. Do integrity dimensions correlate?
4. Are there meaningful interactions between model × constitution?

## Setup

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from scipy.stats import f_oneway, ttest_ind, mannwhitneyu, kruskal, chi2_contingency
from itertools import combinations

# Set style
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (12, 6)

# Load data
EXPERIMENT_ID = "exp_20251023_105245"
BASE_DIR = Path("..")
RESULTS_DIR = BASE_DIR / "results" / "runs" / EXPERIMENT_ID

test_results = []
for result_file in RESULTS_DIR.glob("*_result.json"):
    with open(result_file) as f:
        result = json.load(f)
        test_results.append({
            'trial_id': result['trial_id'],
            'scenario_id': result['scenario_id'],
            'constitution_id': result['constitution_id'],
            'model_id': result['model_id'],
            'factual_adherence': result['integrity_evaluation']['factual_adherence'],
            'value_transparency': result['integrity_evaluation']['value_transparency'],
            'logical_coherence': result['integrity_evaluation']['logical_coherence'],
            'overall_score': result['integrity_evaluation']['overall_integrity_score']
        })

df = pd.DataFrame(test_results)
print(f"Loaded {len(df)} test results for statistical analysis")

## Hypothesis 1: Model Differences Are Significant

**Null Hypothesis (H0):** All models have the same mean integrity score  
**Alternative (H1):** At least one model differs significantly

We'll use ANOVA (parametric) and Kruskal-Wallis (non-parametric) tests.

In [ ]:
# Test normality assumption
print("Normality Tests by Model (Shapiro-Wilk):")
print("="*60)

for model in df['model_id'].unique():
    model_scores = df[df['model_id'] == model]['overall_score']
    stat, p = stats.shapiro(model_scores)
    print(f"{model:30s} p={p:.4f} {'(Normal)' if p > 0.05 else '(Non-normal)'}")

print("\nNote: If p > 0.05, data is approximately normal")

In [ ]:
# ANOVA test
model_groups = [df[df['model_id'] == model]['overall_score'].values for model in df['model_id'].unique()]
f_stat, p_value_anova = f_oneway(*model_groups)

print("One-Way ANOVA (Parametric Test):")
print("="*60)
print(f"F-statistic: {f_stat:.4f}")
print(f"p-value: {p_value_anova:.6f}")
print(f"\nResult: {'REJECT H0' if p_value_anova < 0.001 else 'FAIL TO REJECT H0'}")
print(f"Interpretation: Model differences are {'statistically significant' if p_value_anova < 0.001 else 'not significant'}")

In [ ]:
# Kruskal-Wallis test (non-parametric alternative)
h_stat, p_value_kw = kruskal(*model_groups)

print("Kruskal-Wallis Test (Non-Parametric):")
print("="*60)
print(f"H-statistic: {h_stat:.4f}")
print(f"p-value: {p_value_kw:.6f}")
print(f"\nResult: {'REJECT H0' if p_value_kw < 0.001 else 'FAIL TO REJECT H0'}")

### Pairwise Model Comparisons

In [ ]:
# Pairwise t-tests with Bonferroni correction
models = df['model_id'].unique()
n_comparisons = len(list(combinations(models, 2)))
alpha_corrected = 0.05 / n_comparisons  # Bonferroni correction

print(f"Pairwise Model Comparisons (Bonferroni-corrected α = {alpha_corrected:.4f})")
print("="*80)

pairwise_results = []

for model1, model2 in combinations(models, 2):
    scores1 = df[df['model_id'] == model1]['overall_score']
    scores2 = df[df['model_id'] == model2]['overall_score']
    
    t_stat, p_value = ttest_ind(scores1, scores2)
    mean_diff = scores1.mean() - scores2.mean()
    
    pairwise_results.append({
        'Model 1': model1,
        'Model 2': model2,
        'Mean Diff': f"{mean_diff:+.2f}",
        'p-value': f"{p_value:.6f}",
        'Significant': 'Yes' if p_value < alpha_corrected else 'No'
    })

pairwise_df = pd.DataFrame(pairwise_results)
pairwise_df

## Hypothesis 2: Bad-Faith vs. Honest Constitution Gap

**Null Hypothesis (H0):** Bad-faith and honest constitutions have equal mean integrity  
**Alternative (H1):** Bad-faith scores significantly lower

In [ ]:
# Split data
bad_faith_scores = df[df['constitution_id'] == 'bad-faith']['overall_score']
honest_scores = df[df['constitution_id'] != 'bad-faith']['overall_score']

print("Bad-Faith vs. Honest Constitutions:")
print("="*60)
print(f"Bad-Faith Mean: {bad_faith_scores.mean():.2f} (n={len(bad_faith_scores)})")
print(f"Honest Mean: {honest_scores.mean():.2f} (n={len(honest_scores)})")
print(f"Gap: {honest_scores.mean() - bad_faith_scores.mean():.2f} points")

In [ ]:
# Two-sample t-test
t_stat, p_value_ttest = ttest_ind(honest_scores, bad_faith_scores, alternative='greater')

print("\nTwo-Sample t-Test (One-Tailed):")
print("="*60)
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value_ttest:.10f}")
print(f"\nResult: {'REJECT H0' if p_value_ttest < 0.001 else 'FAIL TO REJECT H0'}")
print(f"Interpretation: Honest constitutions score {'significantly higher' if p_value_ttest < 0.001 else 'not significantly higher'}")

In [ ]:
# Mann-Whitney U test (non-parametric)
u_stat, p_value_mw = mannwhitneyu(honest_scores, bad_faith_scores, alternative='greater')

print("Mann-Whitney U Test (Non-Parametric):")
print("="*60)
print(f"U-statistic: {u_stat:.4f}")
print(f"p-value: {p_value_mw:.10f}")
print(f"\nResult: {'REJECT H0' if p_value_mw < 0.001 else 'FAIL TO REJECT H0'}")

In [ ]:
# Effect size (Cohen's d)
pooled_std = np.sqrt(((len(honest_scores) - 1) * honest_scores.std()**2 + 
                      (len(bad_faith_scores) - 1) * bad_faith_scores.std()**2) / 
                     (len(honest_scores) + len(bad_faith_scores) - 2))

cohens_d = (honest_scores.mean() - bad_faith_scores.mean()) / pooled_std

print("\nEffect Size (Cohen's d):")
print("="*60)
print(f"d = {cohens_d:.4f}")
print(f"Interpretation: {'Small' if abs(cohens_d) < 0.5 else 'Medium' if abs(cohens_d) < 0.8 else 'Large'} effect size")
print(f"\nNote: d > 0.8 indicates a large practical difference")

## Hypothesis 3: Dimensional Failure in Bad-Faith

Which integrity dimension is most affected by bad-faith reasoning?

In [ ]:
# Compare each dimension
dimensions = ['factual_adherence', 'value_transparency', 'logical_coherence']

print("Dimensional Differences (Bad-Faith vs. Honest):")
print("="*80)

for dim in dimensions:
    bad_faith_dim = df[df['constitution_id'] == 'bad-faith'][dim]
    honest_dim = df[df['constitution_id'] != 'bad-faith'][dim]
    
    gap = honest_dim.mean() - bad_faith_dim.mean()
    t_stat, p_value = ttest_ind(honest_dim, bad_faith_dim, alternative='greater')
    
    print(f"\n{dim.replace('_', ' ').title()}:")
    print(f"  Honest: {honest_dim.mean():.2f}")
    print(f"  Bad-Faith: {bad_faith_dim.mean():.2f}")
    print(f"  Gap: {gap:.2f} points")
    print(f"  p-value: {p_value:.6f}")
    print(f"  Significant: {'Yes' if p_value < 0.001 else 'No'}")

In [ ]:
# Visualization
fig, ax = plt.subplots(figsize=(10, 6))

dim_data = {
    'Factual\nAdherence': [
        df[df['constitution_id'] != 'bad-faith']['factual_adherence'].mean(),
        df[df['constitution_id'] == 'bad-faith']['factual_adherence'].mean()
    ],
    'Value\nTransparency': [
        df[df['constitution_id'] != 'bad-faith']['value_transparency'].mean(),
        df[df['constitution_id'] == 'bad-faith']['value_transparency'].mean()
    ],
    'Logical\nCoherence': [
        df[df['constitution_id'] != 'bad-faith']['logical_coherence'].mean(),
        df[df['constitution_id'] == 'bad-faith']['logical_coherence'].mean()
    ]
}

x = np.arange(len(dim_data))
width = 0.35

honest_means = [v[0] for v in dim_data.values()]
bad_faith_means = [v[1] for v in dim_data.values()]

ax.bar(x - width/2, honest_means, width, label='Honest', color='skyblue')
ax.bar(x + width/2, bad_faith_means, width, label='Bad-Faith', color='salmon')

ax.set_ylabel('Mean Score', fontweight='bold')
ax.set_title('Dimensional Breakdown: Honest vs. Bad-Faith', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(dim_data.keys())
ax.legend()
ax.set_ylim(0, 100)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Hypothesis 4: Honest Constitution Convergence

Do honest constitutions (with different value frameworks) produce similar integrity scores?

In [ ]:
# Filter to honest constitutions
honest_consts = ['balanced-justice', 'harm-minimization', 'community-order', 'self-sovereignty']
df_honest = df[df['constitution_id'].isin(honest_consts)]

# ANOVA on honest constitutions only
honest_groups = [df_honest[df_honest['constitution_id'] == const]['overall_score'].values 
                 for const in honest_consts]

f_stat, p_value = f_oneway(*honest_groups)

print("ANOVA: Honest Constitutions Only")
print("="*60)
print(f"F-statistic: {f_stat:.4f}")
print(f"p-value: {p_value:.6f}")
print(f"\nResult: {'Significant differences exist' if p_value < 0.05 else 'No significant differences'}")
print("\nNote: Small p-value suggests constitutions differ, but the effect size matters more")

In [ ]:
# Calculate spread
const_means = df_honest.groupby('constitution_id')['overall_score'].mean()
spread = const_means.max() - const_means.min()

print("\nHonest Constitution Spread:")
print("="*60)
for const in honest_consts:
    mean = const_means[const]
    print(f"{const:25s}: {mean:.2f}")

print(f"\nRange: {spread:.2f} points")
print(f"\nInterpretation: {'Tight clustering' if spread < 5 else 'Moderate spread' if spread < 10 else 'Wide spread'}")

## Hypothesis 5: Model × Constitution Interaction

Do certain models handle bad-faith better than others?

In [ ]:
# Calculate degradation by model
models = df['model_id'].unique()

degradation_data = []

for model in models:
    honest_score = df[(df['model_id'] == model) & (df['constitution_id'] != 'bad-faith')]['overall_score'].mean()
    bad_faith_score = df[(df['model_id'] == model) & (df['constitution_id'] == 'bad-faith')]['overall_score'].mean()
    degradation = honest_score - bad_faith_score
    
    degradation_data.append({
        'Model': model,
        'Honest Mean': honest_score,
        'Bad-Faith Mean': bad_faith_score,
        'Degradation': degradation,
        'Degradation %': (degradation / honest_score * 100)
    })

degradation_df = pd.DataFrame(degradation_data).sort_values('Degradation', ascending=False)

print("Bad-Faith Degradation by Model:")
print("="*80)
degradation_df.round(2)

In [ ]:
# Visualization
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(degradation_df))
ax.bar(x, degradation_df['Degradation'], color='coral', edgecolor='black')

ax.set_xlabel('Model', fontweight='bold')
ax.set_ylabel('Integrity Degradation (points)', fontweight='bold')
ax.set_title('Bad-Faith Degradation by Model\n(Higher = More Susceptible to Motivated Reasoning)',
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(degradation_df['Model'], rotation=45, ha='right')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Summary: Statistical Conclusions

In [ ]:
print("="*80)
print("STATISTICAL CONCLUSIONS")
print("="*80)

print("\n1. MODEL DIFFERENCES:")
print(f"   - ANOVA p-value: {p_value_anova:.6f} → Models differ significantly")
print(f"   - Top model significantly outperforms bottom model (p < 0.001)")

print("\n2. MOTIVATED REASONING DETECTION:")
print(f"   - t-test p-value: {p_value_ttest:.10f} → Bad-faith scores significantly lower")
print(f"   - Effect size (Cohen's d): {cohens_d:.4f} → Large practical difference")
print(f"   - Mean gap: {honest_scores.mean() - bad_faith_scores.mean():.2f} points")

print("\n3. DIMENSIONAL BREAKDOWN:")
bad_faith_factual = df[df['constitution_id'] == 'bad-faith']['factual_adherence'].mean()
honest_factual = df[df['constitution_id'] != 'bad-faith']['factual_adherence'].mean()
factual_gap = honest_factual - bad_faith_factual
print(f"   - Factual adherence most affected: {factual_gap:.2f} point gap")
print(f"   - Bad-faith primarily degrades factual accuracy, not value statements")

print("\n4. VALUE PLURALISM:")
print(f"   - Honest constitution spread: {spread:.2f} points")
print(f"   - Interpretation: Different value frameworks maintain similar integrity")

print("\n5. MODEL ROBUSTNESS:")
most_robust = degradation_df.iloc[-1]['Model']
least_robust = degradation_df.iloc[0]['Model']
print(f"   - Most robust to bad-faith: {most_robust}")
print(f"   - Least robust: {least_robust}")

print("\n" + "="*80)